In [ ]:
# Inventory Data Analysis - Internship Assignment (Modified & Enhanced)
# Author: Kiran Sumit Dalmiya
# Date: <Enter today's date>

## 🧾 Project Overview
This Jupyter Notebook presents a detailed **Inventory Data Analysis** project completed as part of an internship assignment. The goal is to analyze inventory, purchase, and sales data to derive insights, optimize stock control, and improve decision-making using analytical techniques.

### Objectives
1. Understand and clean raw inventory datasets.
2. Perform **Exploratory Data Analysis (EDA)** to identify key trends.
3. Conduct **ABC Classification** to categorize items by value.
4. Calculate **EOQ (Economic Order Quantity)** and **Reorder Points**.
5. Analyze **Lead Times** to identify supplier performance.
6. Compute **Inventory Turnover and Carrying Costs**.
7. Apply **simple forecasting** to anticipate future demand.
8. Recommend **process improvements** based on the findings.

In [ ]:
# --- Import Libraries ---
import os
import glob
import warnings
warnings.filterwarnings("ignore")

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from statsmodels.tsa.arima.model import ARIMA

## 1️⃣ Importing Required Libraries
We begin by importing Python libraries for data manipulation, visualization, and modeling. Packages like `pandas` and `numpy` handle data operations, while `matplotlib` and `seaborn` create visual insights.

In [ ]:
# --- Configuration ---
DATA_DIR = "data_inventory"
REPORT_DIR = "report_outputs"
os.makedirs(DATA_DIR, exist_ok=True)
os.makedirs(REPORT_DIR, exist_ok=True)

# Dataset Google Drive Folder (from assignment PDF)
DRIVE_FOLDER_URL = "https://drive.google.com/drive/folders/1PEDBGTt4bNA5zp8D3ZFLwNCIJc7cgTbs?usp=sharing"

## 2️⃣ Data Loading
The dataset is hosted on Google Drive (link provided in the assignment). Using `gdown`, we can download all related CSV files automatically.
If direct download fails due to permissions, files can be placed manually in the `data_inventory` folder.

In [ ]:
try:
    import gdown
    print("Attempting to download dataset from Google Drive...")
    gdown.download_folder(DRIVE_FOLDER_URL, output=DATA_DIR, quiet=True, use_cookies=False)
    print("Download complete.")
except Exception as e:
    print("Dataset download failed or gdown not available.")
    print("Please ensure CSVs are present in the 'data_inventory' folder.")

csv_files = glob.glob(os.path.join(DATA_DIR, "*.csv"))
print(f"Found {len(csv_files)} CSV files in {DATA_DIR}")

def safe_read_csv(fp):
    try:
        return pd.read_csv(fp, low_memory=False)
    except Exception as e:
        print(f"Error reading {fp}: {e}")
        return None

def find_csv_by_keyword(keyword_list):
    for f in csv_files:
        name = os.path.basename(f).lower()
        for kw in keyword_list:
            if kw.lower() in name:
                return f
    return None

purchase_price_fp = find_csv_by_keyword(["purchaseprice"])
begin_inventory_fp = find_csv_by_keyword(["beginv"])
end_inventory_fp = find_csv_by_keyword(["endinv"])
invoice_fp = find_csv_by_keyword(["invoice"])
final_purchase_fp = find_csv_by_keyword(["final_purchase"])
final_sales_fp = find_csv_by_keyword(["final_sales"])

PurchasePrice = safe_read_csv(purchase_price_fp)
BegInv = safe_read_csv(begin_inventory_fp)
EndInv = safe_read_csv(end_inventory_fp)
Invoice = safe_read_csv(invoice_fp)
Final_Purchase = safe_read_csv(final_purchase_fp)
Final_Sales = safe_read_csv(final_sales_fp)

## 3️⃣ Data Cleaning
To ensure data consistency, we standardize column names, remove extra spaces, and convert types to numeric or date formats where needed.

In [ ]:
def clean_column_names(df):
    df.columns = [str(c).strip().replace("\n", " ").replace(" ", "_") for c in df.columns]
    return df

for df_name in ['PurchasePrice','BegInv','EndInv','Invoice','Final_Purchase','Final_Sales']:
    df = globals().get(df_name)
    if df is not None:
        globals()[df_name] = clean_column_names(df)

## 4️⃣ Exploratory Data Analysis (EDA)
EDA helps uncover hidden patterns in the dataset. We analyze vendor distribution, sales trends, and top-performing items.

In [ ]:
plt.style.use('default')
sns.set_context("talk")

def top_counts_plot(series, topn=10, title=None, savepath=None):
    top = series.value_counts().nlargest(topn)
    plt.figure(figsize=(10,4))
    sns.barplot(x=top.index.astype(str), y=top.values)
    plt.xticks(rotation=25, ha='right')
    plt.title(title or "Top Counts")
    plt.tight_layout()
    if savepath:
        plt.savefig(savepath, dpi=150)
    plt.show()

if PurchasePrice is not None and 'VendorName' in PurchasePrice.columns:
    top_counts_plot(PurchasePrice['VendorName'], title="Top Vendors by Purchase Records")

## 5️⃣ ABC Classification
Items are categorized into **A**, **B**, and **C** classes based on their contribution to total value:
- **A-items**: top ~80% of total value
- **B-items**: next ~15%
- **C-items**: remaining ~5%

This classification helps prioritize monitoring and control.

In [ ]:
def compute_abc(final_sales_df, cost_df=None):
    df = final_sales_df.copy()
    if 'SalesPrice' in df.columns:
        df['UnitPrice'] = df['SalesPrice']
    elif 'SalesDollars' in df.columns:
        df['UnitPrice'] = df['SalesDollars'] / df['SalesQuantity'].replace(0, np.nan)

    df_grouped = df.groupby('Description').agg({'SalesQuantity':'sum','UnitPrice':'median'})
    df_grouped['AnnualValue'] = df_grouped['SalesQuantity'] * df_grouped['UnitPrice']
    df_grouped = df_grouped.sort_values('AnnualValue', ascending=False)
    df_grouped['CumulativePct'] = 100 * df_grouped['AnnualValue'].cumsum() / df_grouped['AnnualValue'].sum()

    def classify(pct):
        if pct <= 80: return 'A'
        elif pct <= 95: return 'B'
        else: return 'C'

    df_grouped['ABC_Class'] = df_grouped['CumulativePct'].apply(classify)
    return df_grouped.reset_index()

abc_table = compute_abc(Final_Sales, PurchasePrice)
abc_table.to_excel(os.path.join(REPORT_DIR, "ABC_analysis.xlsx"), index=False)
abc_table.head()

## 6️⃣ EOQ and Reorder Point Calculation
**Economic Order Quantity (EOQ)** minimizes total inventory costs by balancing ordering and holding costs.

The **Reorder Point (ROP)** indicates when to place a new order, based on demand and lead time.

In [ ]:
def estimate_average_lead_time(invoice_df):
    if 'PODate' in invoice_df.columns and 'InvoiceDate' in invoice_df.columns:
        temp = invoice_df.dropna(subset=['PODate','InvoiceDate']).copy()
        temp['lead_days'] = (pd.to_datetime(temp['InvoiceDate']) - pd.to_datetime(temp['PODate'])).dt.days
        return int(temp['lead_days'].median())
    return 14

def compute_eoq_and_reorder(df_abc, invoice_df=None, carrying_rate=0.25, service_level=0.95):
    from math import sqrt
    z = 1.645
    S = 100  # assumed order cost
    results = []
    lead_time_days = estimate_average_lead_time(invoice_df)

    for _, r in df_abc.iterrows():
        D = r['SalesQuantity']
        C = r['UnitPrice']
        H = carrying_rate * C
        Q = sqrt((2 * D * S) / H)
        daily_demand = D / 365
        safety_stock = z * (0.2 * daily_demand) * np.sqrt(lead_time_days)
        reorder_point = daily_demand * lead_time_days + safety_stock

        results.append([r['Description'], D, C, Q, reorder_point])

    cols = ['Description','AnnualDemand','UnitCost','EOQ','ReorderPoint']
    return pd.DataFrame(results, columns=cols)

eoq_table = compute_eoq_and_reorder(abc_table, Invoice)
eoq_table.to_excel(os.path.join(REPORT_DIR, "EOQ_Reorder.xlsx"), index=False)
eoq_table.head()

## 7️⃣ Lead Time Analysis
Lead time is the time gap between placing a purchase order and receiving goods. Shorter lead times indicate efficient supply chains.

In [ ]:
if Invoice is not None and 'PODate' in Invoice.columns and 'InvoiceDate' in Invoice.columns:
    Invoice['LeadTime_days'] = (pd.to_datetime(Invoice['InvoiceDate']) - pd.to_datetime(Invoice['PODate'])).dt.days
    plt.figure(figsize=(10,4))
    sns.histplot(Invoice['LeadTime_days'].dropna(), bins=30)
    plt.title("Lead Time Distribution (Days)")
    plt.tight_layout()
    plt.show()

## 8️⃣ Inventory Turnover Analysis
Inventory Turnover = Sales / Average Inventory Value

It shows how efficiently inventory is used. A higher ratio means faster movement, while a low ratio implies excess stock.

In [ ]:
def inventory_turnover(beg_inv_df, end_inv_df, sales_df):
    beg_val = (beg_inv_df['onHand'] * beg_inv_df['Price']).sum()
    end_val = (end_inv_df['onHand'] * end_inv_df['Price']).sum()
    avg_inv = (beg_val + end_val) / 2
    sales_val = sales_df['SalesDollars'].sum()
    ratio = sales_val / avg_inv
    return {'begin_val': beg_val, 'end_val': end_val, 'turnover_ratio': ratio}

turnover = inventory_turnover(BegInv, EndInv, Final_Sales)
print(turnover)

## 9️⃣ Forecasting Future Demand
A simple ARIMA time-series model is used to predict the next 6 months of demand for the top-selling SKU.

In [ ]:
def forecast_sku_sales(sales_df, sku, steps=6):
    s = sales_df[sales_df['Description'] == sku].copy()
    s['SalesDate'] = pd.to_datetime(s.iloc[:,0], errors='coerce')
    monthly = s.set_index('SalesDate').resample('M')['SalesQuantity'].sum().fillna(0)
    model = ARIMA(monthly, order=(1,1,1))
    fit = model.fit()
    fc = fit.forecast(steps=steps)
    plt.figure(figsize=(10,4))
    plt.plot(monthly, label='History')
    plt.plot(fc, label='Forecast', linestyle='--')
    plt.legend()
    plt.title(f"Sales Forecast for {sku}")
    plt.show()

sku_top = Final_Sales.groupby('Description')['SalesQuantity'].sum().idxmax()
forecast_sku_sales(Final_Sales, sku_top)

## 🔟 Recommendations & Business Insights
Based on the analyses, below are actionable suggestions for inventory management optimization.

In [ ]:
recommendations = [
    "A-class items: Frequent review, maintain tight control, minimal safety stock.",
    "B-class items: Balanced approach between cost and service level.",
    "C-class items: Bulk purchase, less frequent review.",
    "If turnover ratio < 4: reduce slow-moving items.",
    "If turnover ratio > 12: increase safety stock to avoid stockouts."
]

with open(os.path.join(REPORT_DIR, "process_recommendations.txt"), 'w') as f:
    for r in recommendations:
        f.write(r + '\n')

print("\nProcess Recommendations:")
for r in recommendations:
    print("-", r)

## ✅ Final Remarks
This notebook provides a complete analysis pipeline for inventory data.
- Insights are data-driven and industry-aligned.
- Outputs (charts, Excel sheets, and recommendations) are stored in the `report_outputs/` folder.

This version of the assignment is **ready for submission**, structured, and human-readable.

In [ ]:
print("\nNotebook completed successfully. All outputs stored in 'report_outputs/' folder.")